Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [ ]:
data_root = '/home/roman/data/nomnist' # Change me to store data elsewhere
pickle_file = data_root + '/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [ ]:
import numpy as np

def reformat(dataset, labels):
  # Convert to cubical (3D) input but with depth = 1 
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [ ]:
batch_size = 128
patch1_size = 5
depth1 = 6
patch2_size = 6
depth2 = 16
patch3_size = 6
depth3 = 120

num_hidden = 84

tf.reset_default_graph()

graph = tf.Graph()


with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch1_size, patch1_size, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))
  
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch2_size, patch2_size, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
  
  layer2b_weights = tf.Variable(tf.truncated_normal(
      [patch3_size, patch3_size, depth2, depth3], stddev=0.1))
  layer2b_biases = tf.Variable(tf.constant(1.0, shape=[depth3]))

  last_dim = (((image_size + 1) // 2 + 1) // 2 + 1) // 2 
  print(last_dim)
  layer3_weights = tf.Variable(tf.truncated_normal(
      [last_dim * last_dim * depth3, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

In [ ]:
conv = tf.nn.conv2d(tf_train_dataset, layer1_weights, [1, 2, 2, 1], padding='SAME')
conv.shape

In [ ]:
with graph.as_default():  
  # Model.
  def model(data):
    # 'SAME' is zero padding. h, w of strides are at indices 1,2. indices 0,3 have always constant 1. 
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 2000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.748704
Minibatch accuracy: 7.0%
Validation accuracy: 10.6%
Minibatch loss at step 50: 0.823043
Minibatch accuracy: 82.8%
Validation accuracy: 80.4%
Minibatch loss at step 100: 0.641943
Minibatch accuracy: 80.5%
Validation accuracy: 82.8%
Minibatch loss at step 150: 0.463002
Minibatch accuracy: 82.8%
Validation accuracy: 84.0%
Minibatch loss at step 200: 0.738658
Minibatch accuracy: 81.2%
Validation accuracy: 85.1%
Minibatch loss at step 250: 0.455103
Minibatch accuracy: 86.7%
Validation accuracy: 85.5%
Minibatch loss at step 300: 0.508586
Minibatch accuracy: 83.6%
Validation accuracy: 86.2%
Minibatch loss at step 350: 0.370900
Minibatch accuracy: 89.1%
Validation accuracy: 86.2%
Minibatch loss at step 400: 0.366470
Minibatch accuracy: 88.3%
Validation accuracy: 86.8%
Minibatch loss at step 450: 0.312453
Minibatch accuracy: 93.0%
Validation accuracy: 87.2%
Minibatch loss at step 500: 0.512083
Minibatch accuracy: 90.6%
Validation accuracy: 87.5%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
from tensorflow.contrib import layers
from tensorflow.contrib.layers import xavier_initializer

batch_size = 128
patch1_size = 5
depth1 = 16
patch2_size = 5
depth2 = 32
# patch3_size = 6
# depth3 = 120

num_hidden = 84

tf.reset_default_graph()

graph = tf.Graph()

def CreateConvWB(name, patch_size, idepth, odepth):
    w = tf.get_variable(name, shape=(patch_size, patch_size, idepth, odepth), 
                        initializer=xavier_initializer())
    b = tf.Variable(tf.zeros([odepth]))
    
    return w,b
    
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights, layer1_biases = CreateConvWB('layer1', patch1_size, num_channels, depth1)
  layer2_weights, layer2_biases = CreateConvWB('layer2', patch2_size, depth1, depth2)
  
  
  last_dim = (((image_size + 1) // 2 + 1) // 2) # + 1) // 2 
  print(last_dim)
  fc1_weights = tf.get_variable("fc1w", shape=(last_dim * last_dim * depth2, num_hidden), 
                                initializer=xavier_initializer())
  fc1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# Pooling
def layer(input, weights, biases):
  # 'SAME' is zero padding. h, w of strides are at indices 1,2. indices 0,3 have always constant 1.
  conv = tf.nn.conv2d(input, weights, [1, 1, 1, 1], padding='SAME')
  hidden = tf.nn.elu(conv + biases)
  pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
  return pool

with graph.as_default():  
  # Model.
  def model(data):
    pool = layer(data, layer1_weights, layer1_biases)
    pool = layer(pool, layer2_weights, layer2_biases)
    
    shape = pool.get_shape().as_list()
    print(shape)
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.elu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  # optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdagradOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

7
[128, 7, 7, 32]
[10000, 7, 7, 32]
[10000, 7, 7, 32]


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---